In [1]:
import datetime

import pandas as pd
import spacy
import re
import string
import numpy as np
import sys

import seaborn as sns
from matplotlib import cm
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

from spacy.tokens import Token
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk.corpus import stopwords

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn import metrics

from sklearn.model_selection import ShuffleSplit

import gensim
from gensim import corpora, models
from gensim.models.phrases import Phrases, Phraser
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec
import multiprocessing

from sklearn.model_selection import cross_val_score , GridSearchCV,train_test_split

from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

import enchant
pd.set_option('display.max_rows', 500)

In [49]:
#### Importing the file ####
Path="src/"
Filename='IDBasedonSimliarity.csv'
df=pd.read_csv(Path+Filename)

#ID_df=df[df['Interdiscipilinary']]

manualtag=pd.read_csv(Path+'SamplesManualTagger.csv')

In [50]:
## if it is interdiscipilinary then -1 otherwise 0 (Normal data)
manualtag['interdiscipilinary']=True
manualtag.loc[manualtag.apply(lambda x: (pd.isnull(x['Category_2'])) , axis=1),'interdiscipilinary']=False

In [51]:
df_manualtag=manualtag.merge(df[['Translates','PrimaryMatching', 'PrimaryMatchingSim', 'SecMatching',
       'SecMatchingSim', 'ThrdMatching', 'ThrdMatchingSim', 'ID_Categories','Interdiscipilinary']], how='left' ,left_on='Translates', right_on='Translates')
df_manualtag=df_manualtag[pd.isnull(df_manualtag.PrimaryMatching)==False]

In [54]:
df_manualtag[(df_manualtag['interdiscipilinary']==False) & (df_manualtag['Interdiscipilinary'])][['Translates','Category_1', 'Category_2','interdiscipilinary','PrimaryMatching','SecMatching','ThrdMatching','Interdiscipilinary']]

,Translates,Category_1,Category_2,interdiscipilinary,PrimaryMatching,SecMatching,ThrdMatching,Interdiscipilinary
13,Tailoring of polymer systems combined with rea...,Natural Sciences,NaN,False,Natural Sciences,Engineering Sciences,NaN,True
16,Natural nanoparticles and colloids with their ...,Natural Sciences,NaN,False,Natural Sciences,Engineering Sciences,NaN,True
20,The thin-disk laser (TDL) is a highly promisin...,Engineering Sciences,NaN,False,Natural Sciences,Engineering Sciences,NaN,True
23,Organic field effect transistors (OFETs) hold ...,Engineering Sciences,NaN,False,Natural Sciences,Engineering Sciences,NaN,True
42,Ion mobility spectrometers (IMS) are compact d...,Engineering Sciences,NaN,False,Natural Sciences,Engineering Sciences,NaN,True
45,The aim of the transfer project is the concept...,Engineering Sciences,NaN,False,Engineering Sciences,Natural Sciences,NaN,True
46,Since the proposition of the spin as an additi...,Natural Sciences,NaN,False,Natural Sciences,Engineering Sciences,NaN,True


In [53]:
actuals=df_manualtag['interdiscipilinary']
pred=df_manualtag['Interdiscipilinary']

print(metrics.classification_report(actuals,pred, digits=2))

              precision    recall  f1-score   support

       False       0.81      0.78      0.79        32
        True       0.61      0.65      0.63        17

    accuracy                           0.73        49
   macro avg       0.71      0.71      0.71        49
weighted avg       0.74      0.73      0.74        49



In [58]:
df[df.Translates.str.contains("datum", case=False)].Description.tolist()

['The main task of this proposal is the rigorous mathematical treatment of a localized modelling of geophysical fields, the development of improved techniques to derive the space-time structure of the gravity field, the implementation of these improvements in the ITG Gravity field Recovery Software package to determine precise global gravity field models with regional focus from the original level-1 observations and, most important for the various proposals of this SPP1257, the assimilation of the gravity field products into oceanographic, hydrologic, glaciological and geophysical models referred to selected regions. Various mass transport phenomena in the Earth system are regionally bounded and the inhomogeneous mass distributions are reflected in geographically varying gravity field functionals. These functionals cause signals with varying strength and can be detected with high precision by the innovative sensors of the new gravity and altimetric satellites. Therefore, regional space